[![Binder](https://mybinder.org/badge_logo.svg)](https://nbviewer.org/github/Sistemas-Multimedia/Sistemas-Multimedia.github.io/blob/master/contents/gray_SQ/gray_SQ_companded.ipynb)

[![Colab](https://badgen.net/badge/Launch/on%20Google%20Colab/blue?icon=notebook)](https://colab.research.google.com/github/Sistemas-Multimedia/Sistemas-Multimedia.github.io/blob/master/contents/gray_SQ/gray_SQ_companded.ipynb)

# Gray-scale Image compression Using a Companded Uniform Midtread Scalar Quantizer

In [1]:
%%bash
if [ -d "$HOME/repos" ]; then
    echo "\"$HOME/repos\" exists"
else
    mkdir ~/repos
    echo Created $HOME/repos
fi

"/home/vruiz/repos" exists


In [2]:
%%bash
if [ -d "$HOME/repos/scalar_quantization" ]; then
    cd $HOME/repos/scalar_quantization
    echo "$HOME/repos/scalar_quantization ... "
    git pull 
else
    cd $HOME/repos
    git clone https://github.com/vicente-gonzalez-ruiz/scalar_quantization.git
fi

/home/vruiz/repos/scalar_quantization ... 
Ya está actualizado.


In [3]:
%%bash
if [ -d "$HOME/repos/MRVC" ]; then
    cd $HOME/repos/MRVC
    echo "$HOME/repos/MRVC ... "
    git pull 
else
    cd $HOME/repos
    git clone https://github.com/Sistemas-Multimedia/MRVC.git
fi

/home/vruiz/repos/MRVC ... 
Ya está actualizado.


In [4]:
%%bash
if [ -d "$HOME/repos/image_IO" ]; then
    cd $HOME/repos/image_IO
    echo "$HOME/repos/image_IO ... "
    git pull 
else
    cd $HOME/repos
    git clone https://github.com/vicente-gonzalez-ruiz/image_IO.git
fi

/home/vruiz/repos/image_IO ... 
Ya está actualizado.


In [5]:
%%bash
if [ -d "$HOME/repos/information_theory" ]; then
    cd $HOME/repos/image_IO
    echo "$HOME/repos/information_theory ... "
    git pull 
else
    cd $HOME/repos
    git clone https://github.com/vicente-gonzalez-ruiz/information_theory.git
fi

/home/vruiz/repos/information_theory ... 
Ya está actualizado.


In [7]:
#!ln -sf ~/MRVC/src/logging_config.py .
!ln -sf ~/repos/scalar_quantization/quantizer.py .
!ln -sf ~/repos/scalar_quantization/deadzone_quantization.py .
!ln -sf ~/repos/scalar_quantization/companded_quantization.py .
!ln -sf ~/repos/information_theory/distortion.py .
!ln -sf ~/repos/information_theory/information.py .
!ln -sf ~/repos/image_IO/image_1.py .
!ln -sf ~/repos/image_IO/logging_config.py .

In [8]:
try:
    import matplotlib.pyplot as plt
except:
    !pip install matplotlib
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.axes as ax
    #plt.rcParams['text.usetex'] = True
    #plt.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}'] #for \text command
%matplotlib inline

try:
    import scipy
except:
    !pip install scipy
    
try:
    import cv2
except:
    !pip install opencv-python
    !pip install opencv-python-headless # Binder compatibility
    import cv2

try:
    import skimage
except:
    !pip install scikit-image
    import skimage
    
try:
    import colored
except:
    !pip install colored
    import colored

import pylab
import math
import numpy as np
from scipy import signal
import cv2
import os
import companded_quantization as companded
import distortion
#import image_3 as RGB_image
import image_1 as gray_image
import colored
import information

## Configuration

In [10]:
# Prefix of the gray-scale image to be quantized.

home = os.environ["HOME"]
#fn = home + "/repos/MRVC/images/lena_bw/"
#fn = home + "/repos/MRVC/images/Hommer_bw/"
#fn = home + "/repos/MRVC/images/barb_bw/"
#fn = home + "/repos/MRVC/images/boat_bw/"
#fn = home + "/repos/MRVC/images/crowd_bw/"
#fn = home + "/repos/MRVC/images/fear_bw/"
fn = home + "/repos/MRVC/images/goldhill_bw/"
#fn = home + "/repos/MRVC/images/zelda_bw/"
!ls -l {fn}

quantizer = quantizer.Companded_Quantizer

#RGB_image.write = RGB_image.debug_write # faster
#RGB_image.write = RGB_image.write # higher compression

gray_image.write = gray_image.debug_write # faster
#gray_image.write = gray_image.write # higher compression

QSS = 64 # Quantization Step Size

total 216
-rw-r--r-- 1 vruiz vruiz 219838 sep 27 07:20 000.png


NameError: name 'quantizer' is not defined

## Read the image and show it

In [ ]:
img = gray_image.read(fn, 0).astype(np.int16) # 8 bits/components is not sufficient to shift the components to [-128, 127]
print(img.dtype)
print(img.max(), img.min())
gray_image.show(img, fn + "000.png")

## Show some quantizations

In [ ]:
QS = 1024 # Quantization Step

In [ ]:
Q = quantizer.Companded_Quantizer(Q_step=QS, min_val=0, max_val=255)
print(Q.get_decision_levels())
print(Q.get_representation_levels())

In [ ]:
y, k = Q.quan_dequan(img.astype(np.int16) - 128)
print("Used quantization indexes:", np.unique(k))
gray_image.show_normalized(k, f"{quantizer.name} $\\Delta={QS}$")
gray_image.show(y + 128, f"{quantizer.name} $\\Delta={QS}$")
print("MSE =", distortion.MSE(img, y + 128))
print("SSIM =", distortion.SSIM(img, y + 128))
print("entropy =", information.entropy((k + 128).flatten()))

In [ ]:
QS = 512 # Quantization Step

In [ ]:
Q = quantizer.Companded_Quantizer(Q_step=QS, min_val=0, max_val=255)
print(Q.get_decision_levels())
print(Q.get_representation_levels())

In [ ]:
y, k = Q.quan_dequan(img.astype(np.int16) - 128)
print("Used quantization indexes:", np.unique(k))
gray_image.show_normalized(k, f"{quantizer.name} $\\Delta={QS}$")
gray_image.show(y + 128, f"{quantizer.name} $\\Delta={QS}$")
print("MSE =", distortion.MSE(img, y + 128))
print("SSIM =", distortion.SSIM(img, y + 128))
print("entropy =", information.entropy((k + 128).flatten()))

## RD curve

In [ ]:
def RD_curve(img, Q_steps, quantizer):
    points = []
    for Q_step in Q_steps:
        Q = quantizer.Companded_Quantizer(Q_step=Q_step, min_val=0, max_val=255)
        y, k = Q.quan_dequan(img)
        k = k.astype(np.uint8) # Only positive components can be written in an PNG file
        print("Quantization indexes: ", np.unique(k))
        rate = gray_image.write(k, "/tmp/" + str(Q_step) + '_', 0)*8/k.size
        _distortion = distortion.RMSE(img, y)
        points.append((rate, _distortion))
        print(f"q_step={Q_step:>3}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return points

Q_steps = [2**i for i in range(15, -1, -1)] # Quantization steps (simulating bit-plane encoding)
print(Q_steps)

RD_points = RD_curve(img, Q_steps, quantizer)

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RD_points), c='m', marker='x', label=f"{quantizer.name}", linestyle="dotted")
pylab.title(f"Rate/Distortion Performance ({quantizer.name})")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
pylab.legend(loc='upper right')
pylab.show()

In [ ]:
print(quantizer.name)
with open(f"{quantizer.name}_RD_points.txt", 'w') as f:
    for item in RD_points:
        f.write(f"{item[0]}\t{item[1]}\n")

In [ ]:
import time
while True:
    time.sleep(1)